In [1]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from datetime import datetime as dt

In [2]:
brands_df = pd.read_json('brands.json', lines=True)
receipts_df = pd.read_json('receipts.json', lines=True)
users_df = pd.read_json('users.json', lines=True)

In [3]:
# brands_df.to_csv('brands.csv', index=False)
# receipts_df.to_csv('receipts.csv', index=False)
# users_df.to_csv('users.csv', index=False)

### Investigate brands data

In [4]:
cpg_df = pd.json_normalize(brands_df['cpg'], sep='_').add_prefix('cpg_')
brands_df = pd.concat([brands_df.drop('cpg', axis=1), cpg_df], axis=1)

In [5]:
cpg_df.to_csv('cpg_df.csv', index=False)

In [6]:
brands_df.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id_$oid
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


In [7]:
brands_df.dtypes

_id              object
barcode           int64
category         object
categoryCode     object
name             object
topBrand        float64
brandCode        object
cpg_$ref         object
cpg_$id_$oid     object
dtype: object

In [8]:
brands_df.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
cpg_$ref          0
cpg_$id_$oid      0
dtype: int64

In [9]:
len(brands_df)

1167

In [10]:
brands_df['topBrand'].value_counts()

topBrand
0.0    524
1.0     31
Name: count, dtype: int64

In [11]:
brands_df['category'].value_counts()

category
Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Beauty & Personal Care           6
Cleaning & Home Improvement      6
Deli                             6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: count, dtype: int64

In [12]:
brands_df['name'].value_counts().head(100)

name
ONE A DAY® WOMENS                   2
Caleb's Kola                        2
I CAN'T BELIEVE IT'S NOT BUTTER!    2
V8 Hydrate                          2
Health Magazine                     2
                                   ..
Mug Root Beer                       1
Tom's                               1
test brand @1606765579244           1
test brand @1597342520305           1
PROMISE                             1
Name: count, Length: 100, dtype: int64

In [13]:
brands_df['name'].str.contains('test', case=False).value_counts()

name
False    735
True     432
Name: count, dtype: int64

In [14]:
brands_df['brandCode'].value_counts().head(100)

brandCode
                                 35
HUGGIES                           2
GOODNITES                         2
TEST BRANDCODE @1612366146051     1
TEST BRANDCODE @1612366146827     1
                                 ..
TEST BRANDCODE @1604946245499     1
TEST BRANDCODE @1598639199117     1
PULL UPS                          1
TEST BRANDCODE @1605535020629     1
BRISK                             1
Name: count, Length: 100, dtype: int64

In [15]:
brands_df['brandCode'].str.contains('test', case=False).value_counts()

brandCode
False    573
True     360
Name: count, dtype: int64

In [16]:
brands_df[brands_df['name'].str.contains('test', case=False).fillna(False)].isnull().sum()

_id               0
barcode           0
category          0
categoryCode      4
name              0
topBrand        206
brandCode        72
cpg_$ref          0
cpg_$id_$oid      0
dtype: int64

In [17]:
brands_df[brands_df['brandCode'].str.contains('test', case=False).fillna(False)].isnull().sum()

C:\Users\robxc\AppData\Local\Temp\ipykernel_15292\973855833.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  brands_df[brands_df['brandCode'].str.contains('test', case=False).fillna(False)].isnull().sum()


_id               0
barcode           0
category          0
categoryCode      3
name              0
topBrand        171
brandCode         0
cpg_$ref          0
cpg_$id_$oid      0
dtype: int64

Brands data EDA takeaways:
- Brand name and brand code fields contain many values with the word "test". Seems like bad data.
- Lots of nulls for category, category code, top brand, brand code. Most nulls removed after filtering out "test" brands.
- Not sure what item barcode represents for a brand table.

### Investigate receipts data

In [18]:
a_df = pd.json_normalize(receipts_df['createDate']).add_prefix('createDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
b_df = pd.json_normalize(receipts_df['dateScanned']).add_prefix('dateScanned_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
c_df = pd.json_normalize(receipts_df['finishedDate']).add_prefix('finishedDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
d_df = pd.json_normalize(receipts_df['modifyDate']).add_prefix('modifyDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
e_df = pd.json_normalize(receipts_df['pointsAwardedDate']).add_prefix('pointsAwardedDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
f_df = pd.json_normalize(receipts_df['purchaseDate']).add_prefix('purchaseDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
g_df = pd.json_normalize(receipts_df['_id'])

receipts_df = pd.concat([receipts_df.drop(['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate', '_id'], axis=1), 
                         a_df, b_df, c_df, d_df, e_df, f_df, g_df], axis=1)

In [19]:
receipts_df.dtypes

bonusPointsEarned                 float64
bonusPointsEarnedReason            object
pointsEarned                      float64
purchasedItemCount                float64
rewardsReceiptItemList             object
rewardsReceiptStatus               object
totalSpent                        float64
userId                             object
createDate_$date           datetime64[ns]
dateScanned_$date          datetime64[ns]
finishedDate_$date         datetime64[ns]
modifyDate_$date           datetime64[ns]
pointsAwardedDate_$date    datetime64[ns]
purchaseDate_$date         datetime64[ns]
$oid                               object
dtype: object

In [20]:
receipts_df.isnull().sum()

bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
createDate_$date             0
dateScanned_$date            0
finishedDate_$date         551
modifyDate_$date             0
pointsAwardedDate_$date    582
purchaseDate_$date         448
$oid                         0
dtype: int64

In [21]:
receipts_df[receipts_df['rewardsReceiptStatus'] == 'FINISHED'].isnull().sum()

bonusPointsEarned          62
bonusPointsEarnedReason    62
pointsEarned                0
purchasedItemCount          0
rewardsReceiptItemList      2
rewardsReceiptStatus        0
totalSpent                  0
userId                      0
createDate_$date            0
dateScanned_$date           0
finishedDate_$date          0
modifyDate_$date            0
pointsAwardedDate_$date     4
purchaseDate_$date          0
$oid                        0
dtype: int64

In [22]:
len(receipts_df)

1119

In [23]:
receipts_df['rewardsReceiptStatus'].value_counts()

rewardsReceiptStatus
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64

In [24]:
receipts_df[['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']].describe()

,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent
count,544.000000,609.000000,635.00000,684.000000
mean,238.893382,585.962890,14.75748,77.796857
std,299.091731,1357.166947,61.13424,347.110349
min,5.000000,0.000000,0.00000,0.000000
25%,5.000000,5.000000,1.00000,1.000000
50%,45.000000,150.000000,2.00000,18.200000
75%,500.000000,750.000000,5.00000,34.960000
max,750.000000,10199.800000,689.00000,4721.950000


In [25]:
receipts_df[['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']][receipts_df['rewardsReceiptStatus'] == 'FINISHED'].describe()

,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent
count,456.000000,518.000000,518.000000,518.000000
mean,240.839912,544.664865,15.799228,80.854305
std,296.777392,1153.416470,61.041209,337.853875
min,5.000000,0.000000,0.000000,0.000000
25%,5.000000,5.000000,1.000000,1.000000
50%,45.000000,166.800000,2.000000,11.000000
75%,500.000000,750.000000,5.000000,29.000000
max,750.000000,10199.800000,689.000000,4721.950000


In [26]:
receipts_df[['createDate_$date', 'dateScanned_$date', 'finishedDate_$date', 'modifyDate_$date', 'pointsAwardedDate_$date', 
             'purchaseDate_$date']].describe()

,createDate_$date,dateScanned_$date,finishedDate_$date,modifyDate_$date,pointsAwardedDate_$date,purchaseDate_$date
count,1119,1119,568,1119,537,671
mean,2021-01-27 18:09:48.034587904,2021-01-27 18:09:48.034588928,2021-01-19 04:10:05.020589568,2021-01-28 07:14:28.703043584,2021-01-17 21:42:54.361266176,2020-12-21 02:53:24.980625920
min,2020-10-30 13:17:59,2020-10-30 13:17:59,2021-01-03 07:24:10,2021-01-03 07:24:10,2020-10-30 13:18:00,2017-10-29 17:00:00
25%,2021-01-14 11:13:03.690499840,2021-01-14 11:13:03.690499840,2021-01-08 13:22:42.500000,2021-01-14 13:32:25.500000,2021-01-07 10:15:47,2021-01-04 16:00:00
50%,2021-01-29 09:18:22,2021-01-29 09:18:22,2021-01-19 13:13:57.500000,2021-01-29 09:18:47,2021-01-19 12:54:02,2021-01-13 07:18:59
75%,2021-02-07 05:20:13.736999936,2021-02-07 05:20:13.736999936,2021-01-27 09:42:13.500000,2021-02-07 05:20:13.736999936,2021-01-26 17:12:24,2021-01-25 17:12:58
max,2021-03-01 15:17:34.772000,2021-03-01 15:17:34.772000,2021-02-26 14:36:25,2021-03-01 15:17:34.772000,2021-02-26 14:36:25,2021-03-08 09:37:13


Receipts data EDA takeaways:
- Lots of null values for the various rewards points columns. Upon further inspection, most of the nulls are due to the rewards receipt status being unfinished. Among finished rewards receipts, there is a marginal null count.

##### Investigate items within receipts data

In [28]:
items_df = pd.DataFrame()

for i in range(1, len(receipts_df)):
    if isinstance(receipts_df['rewardsReceiptItemList'].iloc[i], list):
        item_row = pd.json_normalize(receipts_df['rewardsReceiptItemList'].iloc[i], sep='_')
        item_row['receipt_id'] = receipts_df['$oid'].iloc[i]
        item_row['transaction_no'] = i
        items_df = pd.concat([items_df, item_row], axis=0)

In [29]:
# items_df.to_csv('items.csv', index=False)

In [30]:
items_df.dtypes

barcode                                object
description                            object
finalPrice                             object
itemPrice                              object
partnerItemId                          object
quantityPurchased                     float64
needsFetchReview                       object
needsFetchReviewReason                 object
pointsNotAwardedReason                 object
pointsPayerId                          object
preventTargetGapPoints                 object
rewardsGroup                           object
rewardsProductPartnerId                object
userFlaggedBarcode                     object
userFlaggedDescription                 object
userFlaggedNewItem                     object
userFlaggedPrice                       object
userFlaggedQuantity                   float64
receipt_id                             object
transaction_no                          int64
originalMetaBriteBarcode               object
originalMetaBriteDescription      

In [31]:
items_df.isnull().sum()

barcode                               3851
description                            381
finalPrice                             174
itemPrice                              174
partnerItemId                            0
quantityPurchased                      174
needsFetchReview                      6128
needsFetchReviewReason                6721
pointsNotAwardedReason                6600
pointsPayerId                         5673
preventTargetGapPoints                6583
rewardsGroup                          5209
rewardsProductPartnerId               4671
userFlaggedBarcode                    6604
userFlaggedDescription                6735
userFlaggedNewItem                    6618
userFlaggedPrice                      6642
userFlaggedQuantity                   6642
receipt_id                               0
transaction_no                           0
originalMetaBriteBarcode              6869
originalMetaBriteDescription          6930
brandCode                             4340
competitorR

In [32]:
len(items_df)

6940

In [33]:
items_df['description'].value_counts()

description
ITEM NOT FOUND                                                         172
KLARBRUNN 12PK 12 FL OZ                                                120
HUGGIES SIMPLY CLEAN PREMOISTENED WIPE FRAGRANCE FREE BAG 216 COUNT     92
MILLER LITE 24 PACK 12OZ CAN                                            90
KLEENEX POP UP RECTANGLE BOX FACIAL TISSUE 2 PLY 8PK 160 CT             87
                                                                      ... 
CEEEZ IT CRCKR                                                           1
GEVALIA KAFFE Caramel Iced Coffee with Almond Milk, 33.8 fl oz           1
HEINZ Cleaning Vinegar, 1 GAL                                            1
ORE-IDA Diced Hash Brown Potatoes 5 lb. Bag                              1
CAPRI SUN Juice Drink, Variety Pack - 40 pack, 6 fl oz pouches           1
Name: count, Length: 1889, dtype: int64

In [48]:
items_df['brandCode'].value_counts().head(50)

brandCode
HY-VEE                       291
BEN AND JERRYS               180
PEPSI                         93
KROGER                        89
KLEENEX                       88
KNORR                         79
DORITOS                       77
BORDEN                        71
KRAFT                         60
DOLE                          53
FOLGERS                       38
PRINGLES                      29
HIDDEN VALLEY                 29
NATURE'S PATH ORGANIC         28
LAURA'S LEAN BEEF             27
KELLOGG'S                     27
LIGHT & FIT GREEK             26
JUST BARE                     25
HILLSHIRE FARM                24
MCCORMICK GRILL MATES         24
MARIE CALLENDER'S             23
KASHI                         23
TOSTITOS                      23
BRAND                         22
BUSH'S BEST                   22
BIGELOW                       20
BETTY CROCKER                 19
HEMPLER'S                     19
PRIVATE SELECTION             19
CHEESE                        18


Items data EDA takeaways:
- More than half of items missing barcodes. Some "Item Not Found" values.
- Transaction data is semi-structured and most fields are null. Consider storing in NoSQL database or keep as JSON string.

### Investigate users data

In [34]:
created_df = pd.json_normalize(users_df['createdDate']).add_prefix('createdDate_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)
login_df = pd.json_normalize(users_df['lastLogin']).add_prefix('lastLogin_').iloc[:, 0].apply(lambda x: dt.fromtimestamp(x / 1000) if not np.isnan(x) else x)

users_df = pd.concat([users_df.drop(['createdDate', 'lastLogin'], axis=1), created_df, login_df], axis=1)

In [35]:
users_df.head()

,_id,active,role,signUpSource,state,createdDate_$date,lastLogin_$date
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,consumer,Email,WI,2021-01-03 07:24:04.800,2021-01-03 07:25:37.858
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,consumer,Email,WI,2021-01-03 07:24:04.800,2021-01-03 07:25:37.858
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,consumer,Email,WI,2021-01-03 07:24:04.800,2021-01-03 07:25:37.858
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,consumer,Email,WI,2021-01-03 07:25:30.554,2021-01-03 07:25:30.597
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,consumer,Email,WI,2021-01-03 07:24:04.800,2021-01-03 07:25:37.858


In [36]:
users_df.dtypes

_id                          object
active                         bool
role                         object
signUpSource                 object
state                        object
createdDate_$date    datetime64[ns]
lastLogin_$date      datetime64[ns]
dtype: object

In [37]:
users_df.isnull().sum()

_id                   0
active                0
role                  0
signUpSource         48
state                56
createdDate_$date     0
lastLogin_$date      62
dtype: int64

In [38]:
len(users_df)

495

In [39]:
users_df['active'].value_counts()

active
True     494
False      1
Name: count, dtype: int64

In [40]:
users_df['role'].value_counts()

role
consumer       413
fetch-staff     82
Name: count, dtype: int64

In [41]:
users_df['signUpSource'].value_counts()

signUpSource
Email     443
Google      4
Name: count, dtype: int64

In [42]:
users_df['state'].value_counts()

state
WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: count, dtype: int64

In [43]:
users_df[['createdDate_$date', 'lastLogin_$date']].describe()

,createdDate_$date,lastLogin_$date
count,495,433
mean,2020-08-05 17:37:20.606103040,2021-01-22 23:48:08.892304640
min,2014-12-19 06:21:22.381000,2018-05-07 10:23:40.003000
25%,2021-01-04 11:30:17.483500032,2021-01-08 10:14:53.928000
50%,2021-01-13 12:19:38.720999936,2021-01-21 05:57:48.697999872
75%,2021-01-25 09:31:59.408999936,2021-02-03 07:34:11.043000064
max,2021-02-12 06:11:06.240000,2021-03-05 08:52:23.204000


Users data EDA takeaways:

- There are some nulls for last login, signup source, and state, but otherwise no significant data issues.